In [1]:
from sample_entry import samples_sheet, metadata_sheet, work_list, check_list, cell, plans
from IPython.display import display

In [2]:
%run startup.py
print('Sample List')
display(samples_sheet)
print('\nWork List')
display(metadata_sheet)

Sample List


Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value=' '), Cell(column_e…


Work List


Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, style={'backgroundColor': 'grey'}, typ…

In [ ]:
RE(plans)

In [ ]:
RE.resume()